In [9]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display
from hashlib import sha256



In [10]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_ENTRADA = os.getenv("DB_ENTRADA")
DB_SAI = os.getenv("DB_SAI")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

ENTRADA = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_ENTRADA}"
SAIDA = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SAI}'

engine_entrada = create_engine(ENTRADA)
engine_saida = create_engine(SAIDA)


query = "SELECT * FROM Administrador"
dados_originais_df = pd.read_sql(query, engine_entrada)
df_anon_admin = dados_originais_df.copy()


In [11]:
# pseudonimizar telefone 
def pseudonimizar_telefone(telefone):
    s = str(telefone)
    if len(s) != 11 or not s.isdigit():
        return telefone  
    return "**" + s[2:5] + "****" + s[9:11]

df_anon_admin["telefone"] = df_anon_admin["telefone"].apply(pseudonimizar_telefone)


print("==== Dados anonimizados ====")
display(df_anon_admin[["telefone"]])


==== Dados anonimizados ====


,telefone
0,**987****21
1,**976****10
2,**965****09
3,**954****98
4,**943****87
5,**932****76
6,**921****65
7,**910****54
8,**999****66
9,**988****55


In [12]:
# Anonimização administrador
def anonimizar_cpf(cpf, chave):
    cpf = str(cpf)
    chave = str(chave).lower()
    senha_cripto = ''
    i = 0
    for l in cpf:
        deslocamento = ord(chave[i % (len(chave))]) - ord('a')
        nova_letra = chr((ord(l) - 32 + deslocamento) % 95 + 32)
        senha_cripto += nova_letra
        i += 1
    return senha_cripto


chave = "minhachave"
df_anon_admin["cpf"] = df_anon_admin["cpf"].apply(lambda x: anonimizar_cpf(x, chave))

print("==== Dados anonimizados ====")
display(df_anon_admin[["cpf"]])


==== Dados anonimizados ====


,cpf
0,=:@;58>8N4=
1,E@D=56:2F4E
2,=9>924:3I<A
3,A=B=68>7M6?
4,EAF?8:>7L8=
5,?;@;46<5J=>
6,C?D?8:@9N7C
7,@<A<57=6K5@
8,D@E@9;82H8A
9,>:?:35;4I;B


In [4]:
import bcrypt

def criptografar_senha(senha):
    
    salt = bcrypt.gensalt()
    hash_senha = bcrypt.hashpw(senha.encode('utf-8'), salt)
    return hash_senha.decode('utf-8')

def verificar_senha(senha_digitada, senha_criptografada):

    return bcrypt.checkpw(senha_digitada.encode('utf-8'), senha_criptografada.encode('utf-8'))

df_anon_admin["senha"] = df_anon_admin["senha"].apply(criptografar_senha)

print("==== Dados anonimizados ====")
display(df_anon_admin[["senha"]])




==== Dados anonimizados ====


,senha
0,$2b$12$YkTo6HohwRbwmzJoiLfWcu5Vz78Eu0USyyEd4dH...
1,$2b$12$liO7FqKP90B9h46ZC1qeEOc87UqQqS1mFEy9R3N...
2,$2b$12$9ccQKco19iuZd55d2V2IU.mbktvGf4p8lTpmmTN...
3,$2b$12$6O7B/3EG0ZS/SXi7P.DYaeCJADZXADgacze0Bfa...
4,$2b$12$0zJcGJa//J643nRGMUg3juIs/gQy75pPwvAyDg1...
5,$2b$12$hTnXmeKqdPhZV34tkRbQpu3bAC4OExZJP4KZCcY...
6,$2b$12$9cAselltER4uliszRJaSvOL6Jjh1COQ2kUwVj8e...
7,$2b$12$wNYTito2nHpKKCabYNzE2.xb0aT8Nxmgg0H1MO3...
8,$2b$12$2VpvhLLfEra.gXIV33ytwu9iW9UTd4W8dkrYtn9...
9,$2b$12$omAWe49vIOyF37ILct4bUuL8BNTpK.BIIdwPCyx...


In [13]:
# Pseudonimização email
def pseudonimizar_email(email):
    email = str(email)
    parte_local, dominio = email.split("@")
  
    if len(parte_local) > 3:
        parte_local_pseudo = parte_local[:2] + "****"
    else:
        parte_local_pseudo = parte_local + "****"
    return parte_local_pseudo + "@" + dominio
df_anon_admin["email"] = df_anon_admin["email"].apply(pseudonimizar_email)

print("==== Dados anonimizados ====")
display(df_anon_admin[["email"]])


==== Dados anonimizados ====


,email
0,sa****@eitruck.com
1,an****@eitruck.com
2,gi****@eitruck.com
3,be****@eitruck.com
4,ve****@eitruck.com
5,ig****@eitruck.com
6,ma****@eitruck.com
7,is****@eitruck.com
8,jo****@eitruck.com
9,da****@eitruck.com


In [14]:
# Pseudonimização nome
def pseudonimizar_nome(nome_completo):
    for i in range(len(nome_completo)):
        nome = "Usuário " + str(i)
        return nome_completo  # mantém o mesmo comportamento

# Preencher a coluna manualmente
nomes_pseudo = []
for i in range(len(df_anon_admin)):
    nomes_pseudo.append(f"Administrador {i+1}")

df_anon_admin["nome_completo"] = nomes_pseudo

# Mostrar resultados
print("==== Dados anonimizados ====")
display(df_anon_admin[["nome_completo"]])



==== Dados anonimizados ====


,nome_completo
0,Administrador 1
1,Administrador 2
2,Administrador 3
3,Administrador 4
4,Administrador 5
5,Administrador 6
6,Administrador 7
7,Administrador 8
8,Administrador 9
9,Administrador 10


In [7]:
dados_originais_df.to_csv('original_administrador.csv')
df_anon_admin.to_csv('pseudo_administrador.csv')

In [15]:
df_anon_admin.to_sql(name='administrador', con=engine_saida, if_exists='replace', index=False)

13